In [2]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("data/data/train.csv")
df_test = pd.read_csv("data/data/test.csv")
df_val = pd.read_csv("data/data/val.csv")

In [3]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [4]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [31]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [32]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [33]:
tokenizer = Tokenizer(num_words=None,
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [34]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [35]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 24s 73ms/step - loss: 0.5562 - accuracy: 0.7021 - val_loss: 0.4935 - val_accuracy: 0.7553
Epoch 2/10
319/319 [==============================] - 24s 75ms/step - loss: 0.2839 - accuracy: 0.8838 - val_loss: 0.5649 - val_accuracy: 0.7392


In [37]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 11ms/step - loss: 0.5923 - accuracy: 0.7282


Test score: 0.5923188924789429
Test accuracy: 0.7282105684280396


In [38]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 54s 163ms/step - loss: 0.5549 - accuracy: 0.7080 - val_loss: 0.4936 - val_accuracy: 0.7543
Epoch 2/10
319/319 [==============================] - 53s 167ms/step - loss: 0.3264 - accuracy: 0.8647 - val_loss: 0.5327 - val_accuracy: 0.7481


In [39]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 41ms/step - loss: 0.5783 - accuracy: 0.7387


Test score: 0.5782686471939087
Test accuracy: 0.7386589050292969


In [40]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 52s 155ms/step - loss: 0.5527 - accuracy: 0.7084 - val_loss: 0.4935 - val_accuracy: 0.7563
Epoch 2/10
319/319 [==============================] - 49s 152ms/step - loss: 0.3179 - accuracy: 0.8685 - val_loss: 0.5432 - val_accuracy: 0.7475


In [41]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 29ms/step - loss: 0.5933 - accuracy: 0.7366


Test score: 0.5932995080947876
Test accuracy: 0.7366309762001038


In [42]:
# rnn + cnn

In [50]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Conv1D(128, 4, activation='relu'))
model.add(MaxPooling1D(4))
model.add(Conv1D(128, 4, activation='relu'))
model.add(MaxPooling1D(3))
model.add(LSTM(64, dropout=0.5))  # Добавляем LSTM слой с dropout
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
319/319 [==============================] - 27s 77ms/step - loss: 0.5563 - accuracy: 0.7013 - val_loss: 0.4879 - val_accuracy: 0.7552
Epoch 2/10
319/319 [==============================] - 25s 78ms/step - loss: 0.2998 - accuracy: 0.8750 - val_loss: 0.5458 - val_accuracy: 0.7472
Epoch 3/10
319/319 [==============================] - 26s 80ms/step - loss: 0.1049 - accuracy: 0.9620 - val_loss: 0.7318 - val_accuracy: 0.7327
Epoch 4/10
45/45 [==============================] - 0s 8ms/step - loss: 0.9859 - accuracy: 0.7200


Test score: 0.9859035611152649
Test accuracy: 0.719966471195221
